# Lectura, Limpieza y consolidación de bases de datos

In [1]:
import pandas as pd
import numpy as np
import re

## Base JCB

In [ ]:
JCB = pd.read_excel('Data/1_BD_JCB_malignidad_QUANTIL.xlsx')

In [3]:
print('Numero de observaciones y variables: ', JCB.shape)
print('Nombres de las variables:\n', JCB.columns.to_list())

Numero de observaciones y variables:  (2580, 7)
Nombres de las variables:
 ['Id', 'Codigo Unico ', 'Edad', 'Validación - firma operador ', 'Diagnóstico - Texto largo', 'Validación – Fecha ', 'Tipo de examen ']


- `Id`: Es un indice. Vamos a unirla con el nombre de la base para trazar el origen de cada observación. 

- `Codigo Unico `, `Edad`, `Validación - firma operador ` y `Validación – Fecha ` son variables útiles unicamente para la validación de la información. Vamos a darles un nombre estandar.

- **No tenemos informacion sobre la fecha de nacimiento del paciente.**

- `Tipo de examen `: Información útil, pero no requerida en la base de entrega. Vamos a darle un nombre estandar.

- `Diagnóstico - Texto largo`: Información central. Además del diagnostico, contiene descripciones macroscopica, microscopica y de inmunohistoquimica. Vamos a separar esa información en columnas distintas.

In [4]:
JCB['origen'] =  'JCB '+ JCB.Id.apply(str)

In [5]:
JCB.rename(columns={'Codigo Unico ':'cod','Edad':'edad', 
                    'Validación - firma operador ':'val_operador',
                    'Diagnóstico - Texto largo':'diagnostico_tot', 'Validación – Fecha ':'val_fecha',
                    'Tipo de examen ':'tipo_examen'
                   },inplace = True)

Vamos a extraer de la variable  `diagnostico_tot`, las variables centrales: `diagnostico`, `micro` y `macro`; también la variable `inmunohistoquimica` cuando esté presente.

In [6]:
espacios_repetidos= re.compile('(?P<s>\s)(?P=s)*')

var = '('+'|'.join(['DIAGN','MACROS','MICROS', 'INMUNOH'])+')'
pre = r'^\s?(?:(?:D?ESC|ESTU)\w+ )?(?:DE )?'
pos = r'\w+:?\s'
pat = re.compile(pre+var+pos, re.I | re.M)

def separar (texto):
    """Crea un diccionario con los patrones extraidos del texto y el texto entre ellos."""
    #print("\n\nProcesando...\n")
    texto= espacios_repetidos.sub(r'\g<s>',texto) # quitar espacios repetidos
    ms = pat.finditer(texto)
    ll,l,s,t = [''],['PRE'],[0],[]
    for m in ms:
        #print("He encontrado", m.group(0))
        ll.append(m.group(0))
        l.append(m.group(1).upper())
        s.append(m.end())
        t.append(m.start())
    t.append(len(texto))
    k= l +[x+"_Label" for x in l]
    d=dict.fromkeys(k, "")
    for i in range(len(s)):
        d[l[i]+"_Label"] +='\t'+ll[i]
        d[l[i]] += '\n'+texto[s[i]:t[i]]
    return d


In [7]:
JCB['sep_diag'] = JCB.diagnostico_tot.apply(lambda x : separar(str (x)))

In [8]:
JCB['micro']= JCB.sep_diag.apply(lambda x: x.get('MICROS'))
JCB['micro_L']= JCB.sep_diag.apply(lambda x: x.get('MICROS_Label'))
#JCB.micro_L.value_counts(dropna=False)

In [9]:
JCB['macro']= JCB.sep_diag.apply(lambda x: x.get('MACROS'))
JCB['macro_L']= JCB.sep_diag.apply(lambda x: x.get('MACROS_Label'))
#JCB.macro_L.value_counts(dropna=False)

In [10]:
JCB['diagnostico']= JCB.sep_diag.apply(lambda x: x.get('DIAGN',"")+'\n'+x.get('PRE',""))
JCB['diagnostico_L']= JCB.sep_diag.apply(lambda x: x.get('DIAGN_Label'))
#JCB.diagnostico_L.value_counts(dropna=False)

In [11]:
JCB['inmunohistoquimica']= JCB.sep_diag.apply(lambda x: x.get('INMUNOH',""))
JCB['inmunohistoquimica_L']= JCB.sep_diag.apply(lambda x: x.get('INMUNOH_Label'))
#JCB.inmunohistoquimica_L.value_counts(dropna=False)

In [12]:
NotRead = JCB[JCB.micro_L.isna() & JCB.macro_L.isna()]
print('No hemos encontrado micro ni macro en registros.', NotRead.shape)

No hemos encontrado micro ni macro en registros. (466, 17)


**Por ahora vamos a pegar inmunohistoqumica a la micro.**

In [13]:
JCB.inmunohistoquimica = '\nINMUNOHISTOQUIMICA:\n'+JCB.inmunohistoquimica
JCB.micro += JCB.inmunohistoquimica.fillna('')
JCB= JCB.drop(['Id','macro_L','micro_L','inmunohistoquimica', 'inmunohistoquimica_L','sep_diag','diagnostico_tot','diagnostico_L'],1)

## Base ERP

In [14]:
ERP2014 = pd.read_excel('Data/2_BD_ERP_malignidad_ QUANTIL.xlsx', sheet_name='ERP 2014')
ERP2014['origen'] = 'ERP2014 '+ERP2014.ID.apply(str)
ERP2015 = pd.read_excel('Data/2_BD_ERP_malignidad_ QUANTIL.xlsx', sheet_name='ERP 2015')
ERP2015['origen'] = 'ERP2015 '+ERP2015.ID.apply(str)
ERP2016 = pd.read_excel('Data/2_BD_ERP_malignidad_ QUANTIL.xlsx', sheet_name='ERP 2016')
ERP2016['origen'] = 'ERP2016 '+ERP2016.ID.apply(str)
ERP2017 = pd.read_excel('Data/2_BD_ERP_malignidad_ QUANTIL.xlsx', sheet_name='ERP 2017')
ERP2017['origen'] = 'ERP2017 '+ERP2017.ID.apply(str)
ERP2018 = pd.read_excel('Data/2_BD_ERP_malignidad_ QUANTIL.xlsx', sheet_name='ERP 2018')
ERP2018['origen'] = 'ERP2018 '+ERP2018.ID.apply(str)

In [15]:
ERP = pd.concat([ERP2014,ERP2015,ERP2016,ERP2017,ERP2018]).reset_index(drop = True)

In [16]:
print('Numero de observaciones y variables: ', ERP.shape)
print('Nombres de las variables:\n', ERP.columns.to_list())

Numero de observaciones y variables:  (11061, 10)
Nombres de las variables:
 ['ID', 'Codigo Unico', 'FechaNacimiento', 'Tipo de examen', 'Macro - Texto largo', 'Micro - Texto largo', 'Diagnóstico - Texto largo', 'Validación - firma operador', 'Validación - Fecha', 'origen']


- `ID`: Es un indice. Vamos a unirla con el nombre de la base para trazar el origen de cada observación.

- `Codigo Unico `, `FechaNacimiento`, `Validación - firma operador` y `Validación – Fecha` son variables útiles unicamente para la validación de la información. Vamos a darles un nombre estandar.

- No tenemos la edad del paciente pero podemos calcularla apartir de las fechas de validación y de nacimiento.

- `Tipo de examen `: Información útil, pero no requerida en la base de entrega. Vamos a darle un nombre estandar.

- `Macro - Texto largo`, `Micro - Texto largo``Diagnóstico - Texto largo`: Información central. Vamos a darle un nombre estandar.

In [17]:
ERP = ERP.drop('ID',1)
ERP.rename(columns={'Codigo Unico':'cod',
                    'Validación - firma operador':'val_operador',
                    'Macro - Texto largo':'macro',
                    'Micro - Texto largo':'micro',
                    'Diagnóstico - Texto largo':'diagnostico',
                    'Validación - Fecha':'val_fecha',
                    'FechaNacimiento':'fecha_nacimiento',
                    'Tipo de examen':'tipo_examen'
                   },inplace = True)

In [18]:
ERP['edad']= (ERP.val_fecha - ERP.fecha_nacimiento)//np.timedelta64(1,'Y')

## Base PATHOX

In [19]:
Pathox2018 = pd.read_excel('Data/3_BD_PATHOX 2018_malignidad_QUANTIL.xlsx')
Pathox2018['origen'] = 'Pathox2018 '+Pathox2018.ID.apply(str)
Pathox2019 = pd.read_excel('Data/4_BD_PATHOX 2019_malignidad_QUANTIL.xlsx')
Pathox2019['origen'] = 'Pathox2019 '+Pathox2019.ID.apply(str)

In [20]:
Pathox = pd.concat([Pathox2018 ,Pathox2019 ]).reset_index(drop =True)

In [21]:
print('Numero de observaciones y variables: ', Pathox.shape)
print('Nombres de las variables:\n', Pathox.columns.to_list())

Numero de observaciones y variables:  (8681, 14)
Nombres de las variables:
 ['Nacido el', 'Edad del paciente durante la toma de muestra (solo edad)', 'Tipo de examen', 'Macro - Texto largo', 'Micro - Texto largo', 'Inmunohistoquímica- Texto largo', 'Diagnóstico - Texto largo', 'Validación - Operador', 'Especialidad', 'Validación - Fecha', 'Addendum1 - Texto largo', 'Codigo Unico ', 'ID', 'origen']


- `ID`: Es un indice. Vamos a unirla con el nombre de la base para trazar el origen de cada observación.

- `Codigo Unico`, `Nacido el`, `Edad del paciente durante la toma de muestra (solo edad)`, `Validación - Operador` y `Validación - Fecha` son variables útiles unicamente para la validación de la información. Vamos a darles un nombre estandar.

- `Tipo de examen`: Información útil, pero no requerida en la base de entrega. Vamos a darle un nombre estandar.

- `Especialidad`: Información útil, pero presente únicamente en esta base. **Por ahora propongo pegar esta información al diagnostico**

- `Inmunohistoquímica- Texto largo` y `Addendum1 - Texto largo`: Información útil, pero presente unicamente en esta base. **Por ahora propongo pegar el addendum al diagnostico y la inmunohistoquimica a micro**

- `Macro - Texto largo`, `Micro - Texto largo` y `Diagnóstico - Texto largo`: Información central. Vamos a darle un nombre estandar.

In [22]:
Pathox = Pathox.drop('ID',1)
Pathox.rename(columns={'Codigo Unico ':'cod', 
                       'Validación - Operador':'val_operador',
                       'Macro - Texto largo':'macro',
                       'Micro - Texto largo':'micro',
                       'Diagnóstico - Texto largo':'diagnostico',
                       'Validación - Fecha':'val_fecha',
                       'Nacido el':'fecha_nacimiento',
                       'Edad del paciente durante la toma de muestra (solo edad)':'edad',
                       'Tipo de examen':'tipo_examen',
                       'Inmunohistoquímica- Texto largo':'inmunohistoquimica',
                       'Addendum1 - Texto largo':'addendum'
                       },inplace = True)

In [23]:
Pathox.inmunohistoquimica = '\nINMUNOHISTOQUIMICA:\n'+Pathox.inmunohistoquimica
Pathox.micro+= Pathox.inmunohistoquimica.fillna('')

In [24]:
Pathox.addendum ='\nADDENDUM:\n' + Pathox.addendum 
Pathox.Especialidad = "\nESPECIALIDAD "+ Pathox.Especialidad
Pathox.diagnostico += Pathox.addendum.fillna('') + Pathox.Especialidad.fillna('')
Pathox= Pathox.drop(['inmunohistoquimica','addendum','Especialidad'],1)

## Base consolidada

In [25]:
BEntrega = pd.concat([JCB, ERP, Pathox], sort=True).reset_index(drop =True)

In [26]:
print('Numero de observaciones y variables: ', BEntrega.shape)
print('Nombres de las variables:\n', BEntrega.columns.to_list())

Numero de observaciones y variables:  (22322, 10)
Nombres de las variables:
 ['cod', 'diagnostico', 'edad', 'fecha_nacimiento', 'macro', 'micro', 'origen', 'tipo_examen', 'val_fecha', 'val_operador']


In [27]:
BEntrega.isna().apply(lambda x: x.value_counts())

,cod,diagnostico,edad,fecha_nacimiento,macro,micro,origen,tipo_examen,val_fecha,val_operador
False,22322.0,22320,22285,19742,21693,18321,22322.0,22322.0,22322.0,22314
True,NaN,2,37,2580,629,4001,NaN,NaN,NaN,8


In [28]:
BEntrega.fillna({'micro':'', 'macro':'','diagnostico':''}, inplace = True)

In [29]:
BEntrega.cod = BEntrega.cod.apply(str)
BEntrega= BEntrega.astype({'cod':'string', 'diagnostico':'string', 'edad':'Int32', 'fecha_nacimiento':'datetime64', 'macro':'string',
                'micro':'string', 'origen':'string', 'tipo_examen':'category', 'val_fecha':'datetime64', 'val_operador':'category'
                 })
col_order = ['cod', 'fecha_nacimiento', 'edad', 'val_fecha', 'val_operador', 'micro', 'macro','diagnostico','tipo_examen', 'origen']

In [30]:
BEntrega = BEntrega.reindex(col_order, axis='columns')
BEntrega.set_index('origen', inplace = True)

In [31]:
BEntrega.to_pickle('Outputs/df_completa.p', 'gzip')

## Casos piloto

In [32]:
piloto = pd.read_excel('Data/plantilla_casos_piloto.xlsx', header=1)

In [33]:
BPiloto = BEntrega[BEntrega.cod.isin( piloto['Código único'].apply(str))]

In [34]:
BPiloto.to_pickle('Outputs/df_piloto.p', 'gzip')

In [35]:
print('Numero de observaciones y variables: ', BPiloto.shape)
print('Nombres de las variables:\n', BPiloto.columns.to_list())

Numero de observaciones y variables:  (140, 9)
Nombres de las variables:
 ['cod', 'fecha_nacimiento', 'edad', 'val_fecha', 'val_operador', 'micro', 'macro', 'diagnostico', 'tipo_examen']
